# Objective of the project

Using the data given about books, one needs to train a model that predicts a book’s rating.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv("D:\Python Labs\Python Class Project/books.csv", sep=",")

## 1. Examine the data.

First we find out what is the kind of data we have. 

In [3]:
df.head(5)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Unnamed: 12
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,9.78044E+12,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.78044E+12,eng,870,2153167,29221,09-01-2004,Scholastic Inc.,NaN
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.78044E+12,eng,352,6333,244,11-01-2003,Scholastic,NaN
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9.78044E+12,eng,435,2339585,36325,05-01-2004,Scholastic Inc.,NaN
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,439682584,9.78044E+12,eng,2690,41428,164,9/13/2004,Scholastic,NaN


As per my knowledge and a quick internet search Mary GrandPre is designer and not a co-author of Harry Potter as opposed to what is mentioned here. So, let's correct that here.

In [4]:
df.replace(to_replace='J.K. Rowling/Mary GrandPré', value = 'J.K. Rowling', inplace=True)

Let's check the types of columns used in the dataframe.

In [5]:
df.columns.tolist()

['bookID',
 'title',
 'authors',
 'average_rating',
 'isbn',
 'isbn13',
 'language_code',
 '  num_pages',
 'ratings_count',
 'text_reviews_count',
 'publication_date',
 'publisher',
 'Unnamed: 12']

Now, let's find out the number of NA values.

In [6]:
100*df.isna().sum()/len(df)

bookID                 0.000000
title                  0.000000
authors                0.000000
average_rating         0.000000
isbn                   0.000000
isbn13                 0.000000
language_code          0.000000
  num_pages            0.000000
ratings_count          0.000000
text_reviews_count     0.000000
publication_date       0.000000
publisher              0.000000
Unnamed: 12           99.964051
dtype: float64

Thankfully none of the columns except for the last one has no NA values. Now, let's check the data types.

## 2. Checking data format and correcting it.

Now let's check if the data is in the correct format or not.

In [7]:
df.dtypes

bookID                 int64
title                 object
authors               object
average_rating        object
isbn                  object
isbn13                object
language_code         object
  num_pages           object
ratings_count          int64
text_reviews_count     int64
publication_date      object
publisher             object
Unnamed: 12           object
dtype: object

It is clear that only bookID, ratings_count and text_reviews_count are taken as intgers. ISBN, ISBN13 and num_pages should also have been on integers. Lastly, publication_date should have been in datetime format. But, let's take these steps when appropraite later. 

Now, let's try to count the categories of a simple varaible such as language code. It is expected that the variety of this variable would be limited in the table compared to all variables mentioned before it.

In [8]:
df["language_code"].value_counts()

eng            8908
en-US          1408
spa             218
en-GB           214
fre             144
ger              99
jpn              46
mul              19
zho              14
grc              11
por              10
en-CA             7
ita               5
lat               3
enm               3
rus               2
swe               2
ara               1
wel               1
gla               1
tur               1
9.78085E+12       1
nor               1
9.78156E+12       1
glg               1
msa               1
nl                1
9.78159E+12       1
9.78067E+12       1
srp               1
ale               1
Name: language_code, dtype: int64

So there are four 13 digit numbers that seem unnatural for this column. Upon observation, it could be said that they could be ISBN13 entries. So, let's investigate those column.

In [9]:
df.loc[df["language_code"].isin(["9.78085E+12", "9.78156E+12", "9.78159E+12", "9.78067E+12"])]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Unnamed: 12
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner,Jr./Sam B. Warner,3.58,674842111,9.78067E+12,en-US,236,61,6,4/20/2004,Harvard University Press
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net,one of the founding members of this Tolkien w...,3.58,1593600119,9.78159E+12,eng,400,26,4,04-06-2004,Cold Spring Press
5877,22128,Patriots (The Coming Collapse),James Wesley,Rawles,3.63,156384155X,9.78156E+12,eng,342,38,4,1/15/1999,Huntington House Publishers
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown,Son & Ferguson,0,851742718,9.78085E+12,eng,49,0,0,05-01-1977,Brown Son & Ferguson Ltd.


So, it is clear that the entries from the fourth column, that is average rating have all shifted to the left by one column. So, we need to fix this issue before proceeding further.

In [10]:
df.iloc[[3348, 4702, 5877, 8979], 3:13] = df.iloc[[3348, 4702, 5877, 8979], 3:13].shift(periods=-1, axis="columns")

In [11]:
df.iloc[[3348, 4702, 5877, 8979]]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Unnamed: 12
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner,3.58,674842111,9.78067E+12,en-US,236,61,6,4/20/2004,Harvard University Press,NaN
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net,3.58,1593600119,9.78159E+12,eng,400,26,4,04-06-2004,Cold Spring Press,NaN
5877,22128,Patriots (The Coming Collapse),James Wesley,3.63,156384155X,9.78156E+12,eng,342,38,4,1/15/1999,Huntington House Publishers,NaN
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown,0,851742718,9.78085E+12,eng,49,0,0,05-01-1977,Brown Son & Ferguson Ltd.,NaN


Now, let's check if there are any NA values introduced by mistake in the dataframe.

In [12]:
100*df.isna().sum()/len(df)

bookID                  0.0
title                   0.0
authors                 0.0
average_rating          0.0
isbn                    0.0
isbn13                  0.0
language_code           0.0
  num_pages             0.0
ratings_count           0.0
text_reviews_count      0.0
publication_date        0.0
publisher               0.0
Unnamed: 12           100.0
dtype: float64

It is clear that Unnamed: 12 has 100% NA values. So, it is wise to delete the whole coloumn.

In [13]:
del df['Unnamed: 12']

Now, let's try to first change the data type of columns containing numbers specifically.

In [14]:
df = df.astype({"average_rating": float, "  num_pages": int, "ratings_count": int, "text_reviews_count": int})

In [15]:
df.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                 object
language_code          object
  num_pages             int32
ratings_count           int32
text_reviews_count      int32
publication_date       object
publisher              object
dtype: object

For convenince, lets remove the unncessary space in the column name for number of pages in a book.

In [16]:
df = df.rename(columns={"  num_pages": "num_pages"})

Next, let's manipulate the date in the dataframe. It is seen that there are multiple formats in which the months, days and years are seperated. So, first let's put all dates in uniform format convert them into datetime format.

In [17]:
df['publication_date'] = df['publication_date'].apply(lambda x: '{0:0>10}'.format(x))

In [18]:
df['publication_date'] = df['publication_date'].replace('-','/',regex=True)

Here, let's create three new columns from publication_date namely, month, date and year. They may be useful in finding some corelations later.

In [19]:
df[['month', 'date', 'year']] = df['publication_date'].str.split('/', expand=True)

In [20]:
df[['month', 'date', 'year']] = df[['month', 'date', 'year']].astype(int)

In [21]:
df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,month,date,year
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.57,439785960,9.78044E+12,eng,652,2095690,27591,09/16/2006,Scholastic Inc.,9,16,2006
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.49,439358078,9.78044E+12,eng,870,2153167,29221,09/01/2004,Scholastic Inc.,9,1,2004
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.78044E+12,eng,352,6333,244,11/01/2003,Scholastic,11,1,2003
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.56,043965548X,9.78044E+12,eng,435,2339585,36325,05/01/2004,Scholastic Inc.,5,1,2004
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling,4.78,439682584,9.78044E+12,eng,2690,41428,164,09/13/2004,Scholastic,9,13,2004


Now, lets convert the date of publication into a datetime format in case we may have to use it.

In [22]:
df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce', format='%m/%d/%Y')

Now, let's calculate the time lapsed since the publication of the book as Age of the book.

In [23]:
now = pd.to_datetime('now')

C:\Users\ndeob\anaconda3\envs\class_project\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


In [24]:
df['Age_of_book'] = (now.year - df['publication_date'].dt.year) - ((now.month - df['publication_date'].dt.month) < 0)

In [25]:
df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,month,date,year,Age_of_book
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.57,439785960,9.78044E+12,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,9,16,2006,15.0
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.49,439358078,9.78044E+12,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,9,1,2004,17.0
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.78044E+12,eng,352,6333,244,2003-11-01,Scholastic,11,1,2003,18.0
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.56,043965548X,9.78044E+12,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,5,1,2004,18.0
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling,4.78,439682584,9.78044E+12,eng,2690,41428,164,2004-09-13,Scholastic,9,13,2004,17.0


After creating new columns, we will check whether any NaN values are inserted by any chance.

In [26]:
100*df.isna().sum()/len(df)

bookID                0.000000
title                 0.000000
authors               0.000000
average_rating        0.000000
isbn                  0.000000
isbn13                0.000000
language_code         0.000000
num_pages             0.000000
ratings_count         0.000000
text_reviews_count    0.000000
publication_date      0.017974
publisher             0.000000
month                 0.000000
date                  0.000000
year                  0.000000
Age_of_book           0.017974
dtype: float64

In [27]:
df[df['Age_of_book'].isna()]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,month,date,year,Age_of_book
8180,31373,In Pursuit of the Proper Sinner (Inspector Lyn...,Elizabeth George,4.10,553575104,9.78055E+12,eng,718,10608,295,NaT,Bantam Books,11,31,2000,NaN
11098,45531,Montaillou village occitan de 1294 à 1324,Emmanuel Le Roy Ladurie/Emmanuel Le Roy-Ladurie,3.96,2070323285,9.78207E+12,fre,640,15,2,NaT,Folio histoire,6,31,1982,NaN


Since there are only two books which have publication date in ambigious way, it seems that Age_of_book is also compromised. It would be okay to delete just these two rows for final processing.

In [28]:
df = df.drop([8180,11098])

In [29]:
100*df.isna().sum()/len(df)

bookID                0.0
title                 0.0
authors               0.0
average_rating        0.0
isbn                  0.0
isbn13                0.0
language_code         0.0
num_pages             0.0
ratings_count         0.0
text_reviews_count    0.0
publication_date      0.0
publisher             0.0
month                 0.0
date                  0.0
year                  0.0
Age_of_book           0.0
dtype: float64

In [30]:
df

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,month,date,year,Age_of_book
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.57,439785960,9.78044E+12,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,9,16,2006,15.0
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.49,439358078,9.78044E+12,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,9,1,2004,17.0
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.78044E+12,eng,352,6333,244,2003-11-01,Scholastic,11,1,2003,18.0
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.56,043965548X,9.78044E+12,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,5,1,2004,18.0
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling,4.78,439682584,9.78044E+12,eng,2690,41428,164,2004-09-13,Scholastic,9,13,2004,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11122,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9.78156E+12,eng,512,156,20,2004-12-21,Da Capo Press,12,21,2004,17.0
11123,45633,You Bright and Risen Angels,William T. Vollmann,4.08,140110879,9.78014E+12,eng,635,783,56,1988-12-01,Penguin Books,12,1,1988,33.0
11124,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,140131965,9.78014E+12,eng,415,820,95,1993-08-01,Penguin Books,8,1,1993,28.0
11125,45639,Poor People,William T. Vollmann,3.72,60878827,9.78006E+12,eng,434,769,139,2007-02-27,Ecco,2,27,2007,15.0


Now, let's export the clean data into CSV file for further processing.

In [31]:
df.to_csv('book_data_cleaned.csv', index=False)